<a href="https://colab.research.google.com/github/kavyajeetbora/ETL_wages/blob/master/WIOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation and Import

In [1]:
# !pip install -q pyxlsb

In [2]:
%%time
# !wget https://dataverse.nl/api/access/datafile/199104 -O wiot.zip
!wget https://dataverse.nl/api/access/datafile/199097 -O exchange_rates.xlsx
!wget https://dataverse.nl/api/access/datafile/199095 -O sea.xlsx
!wget https://dataverse.nl/api/access/datafile/354108 -O NA_data.xlsx
!wget https://raw.githubusercontent.com/kavyajeetbora/ETL_wages/master/data/PPP_export.csv -O ppp.csv

--2024-07-23 14:05:53--  https://dataverse.nl/api/access/datafile/199097
Resolving dataverse.nl (dataverse.nl)... 194.171.144.89
Connecting to dataverse.nl (dataverse.nl)|194.171.144.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19308 (19K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘exchange_rates.xlsx’

exchange_rates.xlsx 100%[===================>]  18.86K  --.-KB/s    in 0.1s    

2024-07-23 14:05:54 (198 KB/s) - ‘exchange_rates.xlsx’ saved [19308/19308]

--2024-07-23 14:05:54--  https://dataverse.nl/api/access/datafile/199095
Resolving dataverse.nl (dataverse.nl)... 194.171.144.89
Connecting to dataverse.nl (dataverse.nl)|194.171.144.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5536437 (5.3M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘sea.xlsx’

sea.xlsx            100%[===================>]   5.28M  5.49MB/s    in 1.0s    

2024-07-23 14:05:55 (5

In [3]:
import pandas as pd
from zipfile import ZipFile
from tqdm import tqdm
# from pyxlsb import open_workbook as open_xlsb

## SEA

In [4]:
%%time
sea = pd.read_excel("sea.xlsx", sheet_name='DATA')

CPU times: user 12.2 s, sys: 228 ms, total: 12.4 s
Wall time: 17.9 s


In [5]:
sea.head(3)

,country,variable,description,code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AUS,CAP,"Crop and animal production, hunting and relate...",A01,9739.77,13729.03,9726.30,11808.62,11163.98,12429.90,8706.66,11227.29,12556.71,13768.79,18441.27,20106.04,19659.28,18986.06,20525.22
1,AUS,CAP,Forestry and logging,A02,582.69,740.04,540.79,651.96,649.34,676.64,549.27,616.70,716.86,749.91,894.94,923.40,1001.44,1247.96,1206.44
2,AUS,CAP,Fishing and aquaculture,A03,869.31,1122.29,652.69,1024.72,1059.83,923.46,904.15,863.98,1146.37,1108.94,1116.14,1213.94,1182.11,1388.98,1475.22


In [6]:
year_range = list(range(2000,2015))

## Convert to USD $

In [7]:
exchange = pd.read_excel('exchange_rates.xlsx', sheet_name='EXR', header=3)
exchange.columns = list(exchange.columns[:2].str.lower()) + [int(c.replace("_", "")) for c in exchange.columns[2:]]
exchange = pd.melt(exchange, id_vars=['acronym'], value_vars=list(range(2000,2015)), var_name='Year')
exchange.head(3)

,acronym,Year,value
0,AUS,2000,0.582346
1,AUT,2000,0.923600
2,BEL,2000,0.923600


In [8]:
SEA = pd.melt(sea, id_vars=['country','variable','code'], value_vars= year_range, var_name='year')
SEA.head(3)

,country,variable,code,year,value
0,AUS,CAP,A01,2000,9739.77
1,AUS,CAP,A02,2000,582.69
2,AUS,CAP,A03,2000,869.31


In [9]:
SEA_USD = pd.merge(left=SEA,
         right=exchange,
         how='left',
         left_on=SEA['country']+SEA['year'].astype(str),
         right_on=exchange['acronym']+exchange['Year'].astype(str),
         suffixes=('', '_y')
)

SEA_USD['value'] = SEA_USD['value']*SEA_USD['value_y']
SEA_USD = SEA_USD.drop(['key_0', 'value_y', 'acronym', 'Year'], axis=1)

## Unpivot changes
SEA_USD = SEA_USD.pivot(columns=['year'], index=['country','variable', 'code'], values='value').reset_index()
SEA_USD.head(3)

year,country,variable,code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AUS,CAP,A01,5671.916100,7106.640173,5290.163749,7697.850440,8223.153224,9494.504536,6559.162311,9414.767530,10739.314578,10895.298602,16956.729324,20753.655548,20363.082224,18386.613128,18528.013468
1,AUS,CAP,A02,339.327191,383.071345,294.137303,425.002293,478.290208,516.847404,413.792554,517.140569,613.105268,593.406783,822.896435,953.142714,1037.291552,1208.558159,1089.047356
2,AUS,CAP,A03,506.239201,580.937706,355.000049,667.998572,780.648522,705.379380,681.141402,724.499933,980.450138,877.508658,1026.289614,1253.041007,1224.429538,1345.125734,1331.673718


## Adjusted USD $

In [10]:
GO_PI = sea.loc[sea['variable']=='GO_PI'].copy()
GO_PI[GO_PI.columns[4:]] = GO_PI[GO_PI.columns[4:]]/100
GO_PI = pd.melt(GO_PI, id_vars=['country','variable', 'code'], value_vars=year_range, var_name='Year')
GO_PI.head(3)

,country,variable,code,Year,value
0,AUS,GO_PI,A01,2000,0.91576
1,AUS,GO_PI,A02,2000,0.91576
2,AUS,GO_PI,A03,2000,0.91576


In [11]:
SEA = SEA_USD.copy()
SEA = pd.melt(SEA, id_vars=['country','variable','code'], value_vars= year_range, var_name='year')
SEA.head(3)

,country,variable,code,year,value
0,AUS,CAP,A01,2000,5671.916100
1,AUS,CAP,A02,2000,339.327191
2,AUS,CAP,A03,2000,506.239201


In [12]:
adj_USD = pd.merge(left=SEA,
         right=GO_PI,
         how='left',
         left_on=SEA['country']+SEA['year'].astype(str)+SEA['code'],
         right_on=GO_PI['country']+GO_PI['Year'].astype(str)+GO_PI['code'],
         suffixes=('', '_y')
)
adj_USD['value'] = adj_USD['value']*adj_USD['value_y']
adj_USD = adj_USD.drop(['key_0', 'country_y','variable_y','code_y', 'Year', 'value_y'], axis=1)

## Unpivot changes
adj_USD = adj_USD.pivot(columns=['year'], index=['country','variable', 'code'], values='value').reset_index()
adj_USD.sample(4)

year,country,variable,code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
1210,AUT,GO_PI,H53,123.321517,121.294452,124.922968,148.264294,164.809178,143.525329,130.527627,133.799493,145.109887,137.116152,132.570000,157.617386,154.363212,161.668677,163.775925
19271,IND,II,C17,2554.425677,2831.903803,3109.873852,3933.883695,4809.708660,5470.063389,6308.785680,8384.740792,9366.030119,8961.481338,12597.985608,15555.855060,14778.368479,15176.524150,16241.340884
33309,SVK,EMP,M71,9.541133,10.635243,11.935185,16.107100,21.563174,23.855017,24.050494,29.497916,36.744808,38.724579,32.241024,38.309151,33.690116,35.466226,32.374803
17380,HUN,GO_QI,C30,0.119337,0.129249,0.213847,0.302321,0.373754,0.371534,0.379902,0.546529,0.756019,0.437531,0.482636,0.577968,0.476227,0.521371,0.686493


## Industry PPP in Adjusted USD $



In [13]:
ppp1 = adj_USD.groupby(['code','variable']).sum().reset_index()
ppp1 = ppp1.drop(['country'], axis=1)
ppp1[ppp1['variable']=='GO'].sample(5)

year,code,variable,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
484,H49,GO,964433.097665,986285.638175,1.046609e+06,1.217502e+06,1.425665e+06,1.612679e+06,1.826523e+06,2.128599e+06,2.526146e+06,2.293297e+06,2.657847e+06,3.131715e+06,3.301394e+06,3.507225e+06,3.752138e+06
516,H51,GO,241794.273450,230839.560479,2.361345e+05,2.709981e+05,3.134391e+05,3.554780e+05,4.029222e+05,4.615850e+05,5.443075e+05,4.560638e+05,5.258589e+05,6.285693e+05,6.684410e+05,7.042871e+05,7.634283e+05
500,H50,GO,179805.584581,185447.128961,1.929764e+05,2.315018e+05,2.888834e+05,3.335064e+05,3.593106e+05,4.446434e+05,5.289409e+05,4.000477e+05,4.584756e+05,5.173684e+05,5.435233e+05,5.652735e+05,6.102463e+05
356,C33,GO,142429.145774,135414.923268,1.404049e+05,1.670882e+05,1.864468e+05,2.003141e+05,2.187331e+05,2.646469e+05,2.977200e+05,2.691100e+05,2.720409e+05,3.083575e+05,3.044682e+05,3.220359e+05,3.346157e+05
868,T,GO,100235.638863,93352.533610,9.588818e+04,1.103208e+05,1.248890e+05,1.334147e+05,1.412662e+05,1.594846e+05,1.796345e+05,1.698229e+05,1.747142e+05,1.890411e+05,1.827506e+05,1.804322e+05,1.820719e+05


## Industry Total in Current Dollars

In [14]:
ppp2 = SEA_USD.groupby(['code','variable']).sum().reset_index()
ppp2 = ppp2.drop(['country'], axis=1)
ppp2[ppp2['variable']=='GO'].sample(5)

year,code,variable,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
292,C28,GO,1.134274e+06,1.080493e+06,1.086031e+06,1.249481e+06,1.472843e+06,1.631344e+06,1.848402e+06,2.203404e+06,2.525278e+06,2.160167e+06,2.378635e+06,2.862805e+06,2.868575e+06,2.973011e+06,3.055924e+06
436,G45,GO,7.618818e+05,7.247638e+05,7.849275e+05,8.761543e+05,9.704118e+05,1.011238e+06,1.063297e+06,1.168457e+06,1.216466e+06,1.067483e+06,1.193210e+06,1.307228e+06,1.291858e+06,1.292421e+06,1.299872e+06
340,C31_C32,GO,5.974844e+05,5.774206e+05,5.921834e+05,6.354212e+05,6.800163e+05,7.650920e+05,8.389435e+05,9.331346e+05,9.734113e+05,8.791747e+05,8.889205e+05,9.852805e+05,1.027229e+06,1.051905e+06,1.075763e+06
372,D35,GO,1.169714e+06,1.257918e+06,1.205399e+06,1.383148e+06,1.577647e+06,1.834608e+06,2.063355e+06,2.426970e+06,2.698386e+06,2.510265e+06,2.765982e+06,3.019361e+06,3.048179e+06,3.204021e+06,3.299157e+06
116,C17,GO,5.101568e+05,4.770160e+05,4.811063e+05,5.227940e+05,5.776873e+05,6.067074e+05,6.522288e+05,7.298513e+05,7.939621e+05,7.268474e+05,7.969820e+05,8.825485e+05,8.703585e+05,8.830080e+05,8.984470e+05


## PPP Ratio

In [15]:
ppp_ratio = ppp2.copy()
ppp_ratio.loc[:,year_range] =  ppp2.loc[:,year_range] / ppp1.loc[:,year_range]
ppp_ratio.sample(5)

year,code,variable,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
193,C22,COMP,1.170150,1.160953,1.161038,1.159727,1.140160,1.104295,1.081832,1.061617,1.020212,1.012286,1.0,0.966582,0.950357,0.925895,0.930462
632,J62_J63,II,1.048227,1.039173,1.038265,1.044506,1.043020,1.043885,1.034684,1.022030,1.004548,1.002162,1.0,0.978650,0.966413,0.947632,0.937273
151,C19,H_EMPE,2.037227,2.096988,2.084087,1.882951,1.576622,1.271805,1.151176,1.105449,0.935954,1.198018,1.0,0.819682,0.777671,0.839559,0.894637
822,P85,GO_QI,1.600727,1.484484,1.401965,1.333829,1.283435,1.235315,1.183734,1.121998,1.046166,1.009336,1.0,0.984360,0.967675,0.953534,0.941845
675,K66,EMPE,1.273564,1.298182,1.285275,1.257000,1.211309,1.200599,1.136107,1.078603,1.042481,1.044137,1.0,0.997270,0.985689,0.936594,0.905625


## Load PPP Exchange Rate


$V_{GDP} / PPP$

In [16]:
PPP = pd.read_csv('ppp.csv')
PPP = pd.melt(PPP, id_vars=['ISO code','Country', 'Variable code', 'Variable name'], value_vars=[str(i) for i in year_range], var_name='year')
PPP.head(3)

,ISO code,Country,Variable code,Variable name,year,value
0,ABW,Aruba,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,4519.512695
1,AGO,Angola,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,37817.11719
2,AIA,Anguilla,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,273.9180298


In [22]:
PPP[PPP['Country']=='India']

,ISO code,Country,Variable code,Variable name,year,value
78,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,2305846.25
269,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2001,2437321.25
460,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2002,2579105.25
651,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2003,2812778.5
842,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2004,3072820.25
1033,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2005,3538031.75
1224,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2006,3942002.25
1415,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2007,4394168.5
1606,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2008,4668518
1797,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2009,4983383


In [20]:
NA_data = pd.read_excel('NA_data.xlsx', sheet_name='Data')
NA_data = NA_data[NA_data['year'].isin(year_range)][['countrycode', 'year', 'v_gdp']]
NA_data.head(3)

,countrycode,year,v_gdp
30,ABW,2000,3514.117257
31,ABW,2001,3601.926718
32,ABW,2002,3640.996725


In [18]:
PPP_exchange = pd.merge(left=PPP,
         right=NA_data,
         how='left',
         left_on=PPP['ISO code']+PPP['year'].astype(str),
         right_on=NA_data['countrycode']+NA_data['year'].astype(str),
         suffixes=('', '_y')
)
PPP_exchange['value'] = PPP_exchange['value'].str.replace(',','').astype(float)
PPP_exchange['value'] = PPP_exchange['v_gdp']/PPP_exchange['value']

PPP_exchange = PPP_exchange.drop(['key_0', 'countrycode','year_y','v_gdp'], axis=1)
PPP_exchange.dropna(inplace=True)
PPP_exchange

,ISO code,Country,Variable code,Variable name,year,value
0,ABW,Aruba,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,0.777543
1,AGO,Angola,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,3.241085
2,AIA,Anguilla,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,1.480888
3,ALB,Albania,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,31.437671
4,ARE,United Arab Emirates,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,1.087202
...,...,...,...,...,...,...
2852,VNM,Viet Nam,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,7333.596077
2853,YEM,Yemen,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,76.196263
2854,ZAF,South Africa,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,5.441281
2855,ZMB,Zambia,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,3.118285


## Industry PPP Exchange Rate

In [26]:
PPP_exchange

,ISO code,Country,Variable code,Variable name,year,value
0,ABW,Aruba,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,0.777543
1,AGO,Angola,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,3.241085
2,AIA,Anguilla,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,1.480888
3,ALB,Albania,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,31.437671
4,ARE,United Arab Emirates,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,1.087202
...,...,...,...,...,...,...
2852,VNM,Viet Nam,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,7333.596077
2853,YEM,Yemen,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,76.196263
2854,ZAF,South Africa,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,5.441281
2855,ZMB,Zambia,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,3.118285


## World Input Output Analysis

In [19]:
# loading the temp.zip and creating a zip object
with ZipFile("wiot.zip", 'r') as zObject:
    zObject.extractall()

FileNotFoundError: [Errno 2] No such file or directory: 'wiot.zip'

In [ ]:
%%time
df = pd.read_excel('WIOT2000_Nov16_ROW.xlsb', engine='pyxlsb')

In [ ]:
df.head(10)

In [ ]:
# industries = df.iloc[5:, :3].reset_index(drop=True).copy()
# industries.columns = ['code', 'industry', 'country']
# print('Total number of industries',industries.shape[0])
# industries

In [ ]:
# value = df.iloc[5:, 4:industries.shape[0]+4].copy()
# value.columns = industries['code']
# value['industry'] = industries['code']
# value['country'] = industries['country']
# value.sample(5)

In [ ]:
# isProductive = ['K64', 'K65', 'K66', 'L68', 'M69_M70', 'N', 'O84', 'T', 'U']

# value[value['industry'].isin(isProductive)].groupby('country').sum().iloc[:,:-1].sum(axis=1)